In [6]:
class Modeling
  def initialize(people_num, possibility_correct)
    @people_num = people_num
    @people_half = half_num(@people_num) # 終了の値や半分の値に用いる
    @possibility_correct = possibility_correct
  end

  # 半分の値を求める
  def half_num(num)
    (num.to_f / 2).ceil
  end

  # 誤差率をもとめる.先にある割合の意見(finish_num)を集めた方を意見集約の結果とするアルゴリズム
  # このアルゴリズムは必要な人数が動的に変化するので、誤差率をfinish_num個の要素にした配列で返す
  def relative_error_array_by_half_opinion(finish_num, possibility_correct) # possibility_correct => 人が正解する確率p
    relative_error = []
    finish_num.times do |t|
      relative_error << ((1 - possibility_correct)**finish_num) * (possibility_correct**t) * combi(finish_num - 1 + t, t)
    end
    relative_error
  end

  # 誤差率をもとめる.people_num人で多数決を行い意見集約の結果とするアルゴリズム
  def relative_error_by_majority_vote(people_num, possibility_correct) # possibility_correct => 人が正解する確率p
    half_num = half_num(people_num)
    relative_error = 0
    half_num.times do |t|
      relative_error += (possibility_correct**t) * ((1 - possibility_correct)**(people_num - t)) * combi(people_num, t)
    end
    relative_error
  end

  # 確率pを変動させて、誤差率εの求める
  def baseline_method1(possibility_correct) # possibility_correct => 人が正解する確率p
    # 終了する人数を求める
    finish_num = @people_half
    # 誤差率を求め、返す
    relative_error_array_by_half_opinion(finish_num, possibility_correct).inject(:+)
  end

  # 確率pを変動させて、誤差率εの求める
  def baseline_method2(possibility_correct)
    # 多数決を行い誤差率を求め、返す
    relative_error_by_majority_vote(@people_num, possibility_correct)
  end

  # 誤差率εを変動させて、必要な人数Xを求める
  def baseline_method3(relative_error)
    # 必要な人数を求める 1,3,5,,,,と増やす
    people_num = 1
    # 終了する人数を求める
    temp_finish_num = half_num(people_num)
    # 必要な平均人数
    average_num = 0
    # 徐々に人数を増やしていき、誤差率を満たす結果の場合ループを終了する
    loop do
      relative_error_array = relative_error_array_by_half_opinion(temp_finish_num, @possibility_correct)
      expected_error = relative_error_array.inject(:+)
      average_num = 0
      relative_error_array.each_with_index do |e, i| # もっと簡単にかけそう
        average_num += e * (i + temp_finish_num) / expected_error
      end
      break if expected_error <= relative_error
      people_num += 2
      temp_finish_num = half_num(people_num)
    end
    # 必要な人数の平均を返す
    average_num
  end

  # 多数決を意見集約のアルゴリズムとする
  def baseline_method4(relative_error)
    # 必要な人数を求める 1,3,5,,,,と増やす
    people_num = 1
    # 徐々に人数を増やしていき、誤差率を満たす結果の場合ループを終了する
    loop do
      expected_error = relative_error_by_majority_vote(people_num, @possibility_correct)
      break if expected_error <= relative_error
      people_num += 2
    end
    # 必要な人数を返す
    people_num
  end

  # コンビネーションの計算
  def combi(n, k)
    k = n - k if 2 * k > n
    return 1 if k == 0
    ((n - k + 1)..n).reduce(&:*) / (1..k).reduce(&:*)
  end
end

require 'rbplotly'

PEOPELE_NUM = 199
POSSIBILITY_CORRECT = 0.7
DELIMITER = 100

model = Modeling.new(PEOPELE_NUM, POSSIBILITY_CORRECT)

# 縦軸 => 誤差率ε, 横軸 => 人の正解する確率p
error_possibility_x_axis = (DELIMITER/2..DELIMITER).to_a

# 縦軸 => 必要な人数X(コスト), 横軸 => 誤差率ε
cost_error_x_axis = (1..DELIMITER).to_a

# 縦軸 => 誤差率ε, 横軸 => 人の正解する確率p, people_numを変動させたグラフを重ねる
error_possibility_model_arr = []
30.times do |t|
  error_possibility_model_arr << Modeling.new(1 + 5 * t, POSSIBILITY_CORRECT)
end
error_possibility_traces_arr = []
error_possibility_model_arr.each do |elem|
  error_possibility_traces_arr << {x: error_possibility_x_axis, y: error_possibility_x_axis.map{ |e| elem.baseline_method1(e.to_f / DELIMITER)}}
end

# 縦軸 => 必要な人数X(コスト), 横軸 => 誤差率ε, possibility_correctを変動させたグラフを重ねる
cost_error_model_arr = []
9.times do |t|
  cost_error_model_arr << Modeling.new(PEOPELE_NUM, (55 + (5.to_f * t)) / 100)
end
cost_error_traces_arr = []
cost_error_model_arr.each do |elem|
  cost_error_traces_arr << {x: cost_error_x_axis, y: cost_error_x_axis.map{ |e| elem.baseline_method3(e.to_f / DELIMITER)}}
end

# プロットする
pl3 = Plotly::Plot.new(data: error_possibility_traces_arr)
pl3.layout.xaxis = {title: 'possibility_correct (%)'}
pl3.layout.yaxis = {title: 'relative_error (%)'}
pl3.show

pl4 = Plotly::Plot.new(data: cost_error_traces_arr)
pl4.layout.xaxis = {title: 'relative_error (%)'}
pl4.layout.yaxis = {title: 'people'}
pl4.show


(pry):736: warning: already initialized constant PEOPELE_NUM
(pry):599: warning: previous definition of PEOPELE_NUM was here
(pry):737: warning: already initialized constant POSSIBILITY_CORRECT
(pry):600: warning: previous definition of POSSIBILITY_CORRECT was here
(pry):738: warning: already initialized constant DELIMITER
(pry):601: warning: previous definition of DELIMITER was here


#<Plotly::Offline::HTML:0x00007fc299d8bd88 @id="36c00fef-7e0d-40e0-a520-99514ff1254d", @data=[{:x=>[50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100], :y=>[0.5, 0.49, 0.48, 0.47, 0.45999999999999996, 0.44999999999999996, 0.43999999999999995, 0.43000000000000005, 0.42000000000000004, 0.41000000000000003, 0.4, 0.39, 0.38, 0.37, 0.36, 0.35, 0.33999999999999997, 0.32999999999999996, 0.31999999999999995, 0.31000000000000005, 0.30000000000000004, 0.29000000000000004, 0.28, 0.27, 0.26, 0.25, 0.24, 0.22999999999999998, 0.21999999999999997, 0.20999999999999996, 0.19999999999999996, 0.18999999999999995, 0.18000000000000005, 0.17000000000000004, 0.16000000000000003, 0.15000000000000002, 0.14, 0.13, 0.12, 0.10999999999999999, 0.09999999999999998, 0.08999999999999997, 0.07999999999999996, 0.06999999999999995, 0.06000000000000005, 0.050000000000000044, 0.040000000000000036, 0.030000000000000027, 0.020000000000000018, 0.010000000000000009, 0.0]}, {:x=>[50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100], :y=>[0.5, 0.4812549994, 0.4625399807999999, 0.44388485419999996, 0.42531938559999993, 0.4068731249999999, 0.3885753343999999, 0.3704549158000001, 0.35254033920000005, 0.33485957060000004, 0.31744000000000006, 0.3003083694, 0.28349070080000005, 0.2670122242, 0.2508973056, 0.23516937499999996, 0.21985085439999996, 0.20496308579999997, 0.19052625919999994, 0.17655934060000006, 0.16308000000000006, 0.15010453940000004, 0.13764782080000004, 0.1257231942, 0.11434242559999999, 0.103515625, 0.09325117439999997, 0.08355565579999999, 0.07443377919999998, 0.06588831059999997, 0.05791999999999997, 0.05052750939999996, 0.04370734080000003, 0.037453764200000024, 0.03175874560000001, 0.02661187500000001, 0.022000294400000002, 0.0179086258, 0.014318899199999995, 0.011210480599999996, 0.008559999999999995, 0.006341279399999994, 0.004525260799999993, 0.0030799341999999934, 0.001970265600000005, 0.0011581250000000031, 0.0006022144000000017, 0.0002579958000000007, 7.761920000000021e-05, 9.850600000000027e-06, 0.0]}, {:x=>[50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100], :y=>[0.5, 0.4729477257149746, 0.4460034731166029, 0.4192742285427514, 0.3928649200495801, 0.36687741917115224, 0.3414095793708294, 0.3165543227701151, 0.29239878619384113, 0.2690235368946895, 0.24650186752000008, 0.2248991789657181, 0.20427245873317543, 0.18466986127225832, 0.16613039556843043, 0.14868372392111326, 0.1323500744781449, 0.11714026864751055, 0.10305586301634226, 0.09008940388240438, 0.07822479096000005, 0.06743774527655501, 0.05769637474514567, 0.04896182940006119, 0.04118903683622163, 0.03432750701904297, 0.028322194350266325, 0.023114403709482194, 0.0186427261637182, 0.01484398917266121, 0.011654205439999989, 0.009009504098156054, 0.00684702769147758, 0.005105778469969017, 0.00372739785098978, 0.0026568635802539085, 0.0018430901570849726, 0.0012394195144127577, 0.0008039907946370431, 0.0004999803724290901, 0.00029570607999999964, 0.0001645929255494754, 8.500149873708868e-05, 3.992476631525706e-05, 1.656411575903363e-05, 5.80134505859378e-06, 1.5898640480469072e-06, 2.9570897681503716e-07, 2.7120741181030537e-08, 4.4254343383480227e-10, 0.0]}, {:x=>[50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100], :y=>[0.5, 0.46860840662958275, 0.43739213811039956, 0.406524002349233, 0.3761718235685524, 0.3464960748529061, 0.31764765684165164, 0.2897658661559746, 0.2629765928965597, 0.23

#<Plotly::Offline::HTML:0x00007fc299d008a0 @id="f105eb2f-4c7b-4d5a-8816-252fcd9a8ccf", @data=[{:x=>[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100], :y=>[531.3374093991372, 413.712211680258, 345.99569990474083, 298.23970987671225, 262.4548481304765, 234.6466023470128, 210.83211939822047, 191.00487574395345, 173.17734166774494, 157.3467504627048, 145.48550241276035, 131.6622573180323, 121.79989045249778, 111.94854773630074, 102.10993775582799, 94.24963963076667, 86.4002158690515, 78.56335435091543, 72.69521006778876, 66.83642644845737, 60.988363911485166, 55.15271613205833, 51.270219430899786, 47.39496912185571, 41.59780790423963, 37.74529501698061, 33.90442553678007, 31.988981207331243, 28.169650545578477, 24.36845752842513, 22.4759676916538, 20.58978749570594, 16.840025282158983, 14.978876961907444, 13.129101207524517, 11.293055658101974, 9.474011917112387, 7.6767380436670045, 7.6767380436670045, 5.908662653755561, 4.182530795072788, 4.182530795072788, 2.523809523809524, 2.523809523809524, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]}, {:x=>[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100], :y=>[129.2904121668454, 99.504343945868, 83.65412023086859, 71.79054228081004, 63.89675004851813, 56.01866141756137, 50.12294746451157, 46.19992334470824, 42.283949141189446, 38.37621858476936, 34.47825014258204, 32.533520962649035, 28.65409333312963, 26.720158978877237, 24.790703529824796, 22.866311747340575, 20.9476924719564, 19.035717418996462, 17.13147683297932, 15.236361832524636, 15.236361832524636, 13.352190820937228, 11.481412428187713, 11.481412428187713, 9.627449937787796, 9.627449937787796, 7.795328826060233, 7.795328826060233, 5.992932862190813, 5.992932862190813, 5.992932862190813, 4.233870967741936, 4.233870967741936, 4.233870967741936, 4.233870967741936, 2.5454545454545454, 2.5454545454545454, 2.5454545454545454, 2.5454545454545454, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]}, {:x=>[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100], :y=>[54.62613517372524, 42.77162480520175, 36.86433852171449, 30.976330420017554, 27.065285340646817, 25.11520592797128, 21.22870446374437, 19.293789614722762, 17.365766565309492, 15.445998019676509, 15.445998019676509, 13.53628559253196, 11.639083466233394, 11.639083466233394, 9.757862957843116, 9.757862957843116, 9.757862957843116, 7.897780037735344, 7.897780037735344, 6.067017082785808, 6.067017082785808, 6.067017082785808, 6.067017082785808, 4.279854147675478, 4.279854147675478, 4.279854147675478

#<CZTop::Socket::PUB:0x7fc29b4e8be0 last_endpoint="tcp://127.0.0.1:62983">